### <div style="text-align:center"> Onboard Historical Data </div>

In [1]:
# importing required libraries
import pandas as pd
import mysql.connector
import pymysql
from mysql.connector import errorcode
from sqlalchemy import create_engine
import boto3
import s3fs
import datetime as dt
from datetime import datetime

C:\Users\PC\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Import five fiscal years of Yeshiva University's giving data from July 1, 2015 through June 30, 2020.  The fiscal year runs from July 1st to June 30th.  For example, fiscal year 2016 corresponds to July 1, 2015 to June 30, 2016.

In [2]:
#Insert your path name to your 5_Year_Giving.csv historical data file.  Print the shape of the data set and view the header 
#to make sure the data looks as expected.

df = pd.read_csv (r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/CleanDonorData%20-%20CleanDonorData.csv', encoding='latin-1')
print (df.shape)
df.head()

(57157, 17)


,FakeID,PrimaryAffiliation,Person/Org,City,State,Zip,FiscalYear,ContributionDate,Amount,ContributionType,DesignationCode,DesignationName,VSEPurpose,College,DesignationType,DesignationPurpose,PaymentType
0,8363,Alum,P,West Orange,NJ,7052,2016,7/1/2015,500,Gift,AF08,RIETS Annual Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Credit Card
1,7205,Alum,P,New York,NY,10014,2016,7/1/2015,250,Gift,AF04,Benjamin N. Cardozo School of Law Annual Fund,3A02 - Academic Divisions,Cardozo School of Law,Unrestricted,Annual Fund,Credit Card
2,10976,Parent of Past Student,P,Bronx,NY,10463,2016,7/1/2015,100,Gift,B313,The Jay Stepelman Memorial Award for Excellenc...,NaN,High Schools (combined),Restricted,NaN,Credit Card
3,18472,Friend,P,Great Neck,NY,11023,2016,7/1/2015,10,Gift,GU06,Yeshiva University Hs for Girls General Unrest...,NaN,High Schools (combined),Unrestricted,Annual Fund,Credit Card
4,11833,Alum,P,Forest Hills,NY,11375,2016,7/1/2015,25,Gift,GU06,Yeshiva University Hs for Girls General Unrest...,NaN,High Schools (combined),Unrestricted,Annual Fund,Credit Card


We need to clean up the dataset before we can use it.

Albert Einstein Medical School separated from Yeshiva University in 2015.  We want to remove any gifts to the College Albert Einstein Medical School since they are no longer a part of the university.

In [3]:
#Print the shape of the dataset to ensure that the rows containing Albert Einstein have been removed.

index_names = df[df['College'] == 'Albert Einstein College of Med'].index
df.drop(index_names, inplace = True)
print (df.shape)

(57157, 17)


526 donations to Albert Einstein Medical School were removed.

Change the colleges 'YU Girls High School' and 'YU Boys High School' to 'High Schools (combined)'.

In [4]:
df['College'] = df['College'].replace(['YU Girls High School','YU Boys High School'],
                                    ['High Schools (combined)','High Schools (combined)'])
                                                                                 
df['College'].value_counts()

Yeshiva University               20846
RIETS                            12473
Cardozo School of Law             7622
High Schools (combined)           5969
Center for the Jewish Future      2452
Stern College for Women           2197
Undergraduate                     1187
Ferkauf Grad School of Psych      1065
Yeshiva College                   1029
Wurzweiler School of Soc Work      849
Sy Syms School of Business         425
Yeshiva University Museum          404
Azrieli Grad School of J Edu       231
Bernard Revel Graduate School      185
University-School Partnership       87
Belz School of Jewish Music         67
The Katz School                     53
James Striar School                 16
Name: College, dtype: int64

Combine the college 'Undergraduate Men' with 'Undergraduate'

In [5]:
df['College'] = df['College'].replace(['Undergraduate Men'],['Undergraduate'])
                                                                                 
df['College'].value_counts()

Yeshiva University               20846
RIETS                            12473
Cardozo School of Law             7622
High Schools (combined)           5969
Center for the Jewish Future      2452
Stern College for Women           2197
Undergraduate                     1187
Ferkauf Grad School of Psych      1065
Yeshiva College                   1029
Wurzweiler School of Soc Work      849
Sy Syms School of Business         425
Yeshiva University Museum          404
Azrieli Grad School of J Edu       231
Bernard Revel Graduate School      185
University-School Partnership       87
Belz School of Jewish Music         67
The Katz School                     53
James Striar School                 16
Name: College, dtype: int64

Remove any payment type called "Goods".  This refers to payments collected for goods & services which is a used to cover food at an event and cannot be counted as a charitable donation. 

In [6]:
#Print the shape of the dataset to make sure the "goods" rows were removed.

discard = ["Goods"]
df2 = df[~df['PaymentType'].str.contains('|'.join(discard))]
print (df2.shape)
df2['PaymentType'].value_counts()

(57157, 17)


Credit Card                    25564
Cash or Check                  23615
Lockbox Holding                 3302
Wire Transfer                   3020
Payroll Deduction                628
Matching Gift                    452
Bequest Realized                 242
Assets deposited in advance      154
Stocks and Securities            144
Grants                            32
Paper Credits                      3
Conditional Gift                   1
Name: PaymentType, dtype: int64

407 goods & services payment types were removed from the dataset.

Combine payment types where appropriate. <br>
1) Cash/Check into Cash or Cash.<br> 
2) Credit Card (VISA), Credit Card (Master Card), Credit Card (Discover), Credit Card (American Express) into Credit Card. <br>
3) Bequest Wired into Bequest Realized.<br> 
4) Matching Wire Transfer into Matching Gift. <br>
5) Canadian Friends of YU into Cash or Check. <br>
6) Israel Retained Cash into Cash or Check. <br>
7) Canadian Cash into Cash or Check. <br>
8) Stock into Stocks and Securities. <br>
9) Electronic Funds Transfer into Wire Transfer.<br>

In [7]:
#View the Payment Type counts to see how they have been condensed.

df2['PaymentType'] = df2['PaymentType'].replace(['Cash/Check','Credit Card (VISA)','Credit Card (Master Card)',
                                                 'Credit Card (Discover)','Credit Card (American Express)','Bequest Wired',
                                                 'Matching Wire Transfer','Canadian Friends of YU', 'Israel Retained Cash',
                                                'Canadian Cash','Stock','Electronic Funds Transfer'],
                                                ['Cash or Check','Credit Card','Credit Card','Credit Card','Credit Card',
                                                 'Bequest Realized','Matching Gift','Cash or Check','Cash or Check',
                                                 'Cash or Check','Stocks and Securities','Wire Transfer'])
                                               
                                               
df2['PaymentType'].value_counts()

Credit Card                    25564
Cash or Check                  23615
Lockbox Holding                 3302
Wire Transfer                   3020
Payroll Deduction                628
Matching Gift                    452
Bequest Realized                 242
Assets deposited in advance      154
Stocks and Securities            144
Grants                            32
Paper Credits                      3
Conditional Gift                   1
Name: PaymentType, dtype: int64

We went from 23 payment types to 12.

In [8]:
# renaiming columns for analysis 

df2 = df2.rename(columns={"Person/Org":"PersonOrg"})

# confirming update
df2.head(1)

,FakeID,PrimaryAffiliation,PersonOrg,City,State,Zip,FiscalYear,ContributionDate,Amount,ContributionType,DesignationCode,DesignationName,VSEPurpose,College,DesignationType,DesignationPurpose,PaymentType
0,8363,Alum,P,West Orange,NJ,7052,2016,7/1/2015,500,Gift,AF08,RIETS Annual Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Credit Card


## Sending Dataframe to S3 Bucket (Archive)

In [9]:
# creating a variable that stores date to be append on the file to be archive
today = dt.datetime.today()
date = today.strftime("%Y%m%d")

In [10]:
#connect and load the historical data file to s3

s3pathName = 's3://yu-donation-bucket/'
FileNameFullPath = 'donations' 
filenames3 = "%s%s%s.csv"%(s3pathName,FileNameFullPath,date) #name of the filepath and csv file

df2.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://yu-donation-bucket/donations20210505.csv


## Connect to MySQL Workbench

In [11]:
# credentials for connecting to MySQL

host_name = "database-1.cvt3j4afpbrp.us-east-1.rds.amazonaws.com"
db_name = "staging_db"
u_name = "admin"
pwd = "Joli2016!"

In [19]:
# create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=db_name, user=u_name, pw=pwd))

# connect to server
connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')
print('Connected to database.')
cursor = connection.cursor()

Connected to database.


### Loading Donation Staging Table

In [13]:
# call store procedure to delete data from the donation staging table
cursor.callproc('staging_db.del_donation')
print("Donation staging table (donation_stg) cleared.")

Donation staging table (donation_stg) cleared.


In [20]:
# loading the donation staging table (donation_stg) in the staging database
connection.commit()
df2.to_sql('donation_stg', engine, if_exists='append', index=False)
print("Donation staging table (donation_stg) loaded.")


Donation staging table (donation_stg) loaded.


### Loading Donation Table in the Donation Database

In [21]:
# connect to donation_db
donation_db = "donation_db"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=donation_db, user=u_name, pw=pwd))

connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')

print('Connected to database.')
cursor = connection.cursor()

Connected to database.


In [22]:
# call stored procedures to load dimenstions in donation_db
cursor.callproc('donation_db.load_dim_donation')
print("Donation dimension table (dim_donation) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_date')
print("Date dimension table (dim_date) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_donor')
print("Donor dimension table (dim_donor) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_college')
print("College dimension table (dim_college) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_designation')
print("Designation dimension table (dim_designation) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_loc')
print("Location dimension table (dim_location) in donation database (donation_db) loaded.")

#call stored procedures to load fact table in donation_db
cursor.callproc('donation_db.load_donation_fact')
print("Donation fact table (donation_fact) in donation database (donation_db) loaded.")


Donation dimension table (dim_donation) in donation database (donation_db) loaded.
Date dimension table (dim_date) in donation database (donation_db) loaded.
Donor dimension table (dim_donor) in donation database (donation_db) loaded.
College dimension table (dim_college) in donation database (donation_db) loaded.
Designation dimension table (dim_designation) in donation database (donation_db) loaded.
Location dimension table (dim_location) in donation database (donation_db) loaded.
Donation fact table (donation_fact) in donation database (donation_db) loaded.


In [23]:
# commiting and closing database connection
connection.commit()
connection.close()
print('Disconnected from database.')

Disconnected from database.
